Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [4]:
# Read a few reviews from the "Excellent" category

pd.set_option("display.max_colwidth", 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
228,4568,"\nA malt that was overlooked by one owner (Glenmorangie) and is now, under the auspices of a new one (La Martiniquaise), beginning to ease itself back into the single malt world. Its problem has been the price: too low (see Speyburn). The standard 12 year old bottling gives a perfect idea of its potential. This is as soft and sweet a malt as you could ask for: peaches, toffee, and baked apple, with a tobacco/cedar/pine note from the oak.",0
1674,3857,"\nA solid presence in the range, the strength and malt content continue to mark these out over their contemporaries. Grapefruit peels, sliced juicy orange, and lemon butter icing on the nose. The palate is thinner than others in the range, with crystallized orange, fudge, plus cardamom and mild pepper. The finish of sugared satsumas is boosted by a long lasting trail of spices.",0
831,4022,"\nA blend of Highlands scotch, Canadian whisky, and house-made high-rye whiskey. Aromas of stewed red fruit, pomegranate, and mint hold promise, but the whiskey is quiet, with subtle peach, unripe raspberry, apricots, and white flowers. White chocolate, cornbread, and biscotti flavors add a good deal of sweetness. The finish is pleasant and rounded. An interesting experiment, but it’s hard to discern what each component brings to the table.",0


In [5]:
# Read a few reviews from the "Poor" category

pd.set_option("display.max_colwidth", 0)
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
1272,5100,"\nAll of the elements that work with Southern Shine’s Apple Pie bottling don’t work with their Blueberry. An extremely evasive nose leads to an artificially-flavored palate. Here the blueberry tastes more like flavored bubblegum than fruit. The white pepper from the base spirit clashes with the berry like a multi-car pileup. The finish is hot, too dry, and a little sour. There’s just nothing about this that’s pleasant or enjoyable.",2
86,5044,"\nAn oddball whiskey by all accounts, with a very high 72.2% of wheat in the mashbill, distillation on an old farmhouse cognac still, and aging in a little Oregon barn in 63-gallon new oak barrels. The result is a strange, disjointed whiskey that starts out pillow-soft with wheat and caramel, and then dramatically shifts to raw oak, nougat, and cinnamon stick. The flavors in Billy don’t integrate well: they're like a jazz band severely out of step.",2
2193,5064,"\nWhen they say berry, they mean berries: baskets of blueberries, blackberries, and raspberries, until suddenly the palate veers off into grape popsicles. This is one sweet, unabashedly faux-fruity potation, but other than traces of oak, not much whisky flavor remains. Still, in the right hands it could be a lot of cocktail fun. Rather than serving it in a whisky glass, a dram or two on French vanilla or coconut-milk ice cream is genuinely scrumptious.",2


# Split the Training Set into Train/Validation

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train["description"],
                                                   train["ratingCategory"], 
                                                   test_size=0.2, 
                                                   stratify=train["ratingCategory"], 
                                                   random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words = 'english', ngram_range = (1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), 
                 ('clf', rfc)])

# **SUBMISSION 1**

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [10]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=30, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 30 folds for each of 32 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  6.0min finished


GridSearchCV(cv=30, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        n

In [13]:
from sklearn.metrics import accuracy_score
pred = grid_search.predict(X_test)

accuracy_score(y_test, pred)

0.7053789731051344

In [14]:
grid_search.best_score_

0.7237739268320308

In [15]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 10,
 'vect__max_df': 1.0,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

random_search = RandomizedSearchCV(pipe, parameters, cv=30, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

Fitting 30 folds for each of 10 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.0min finished


RandomizedSearchCV(cv=30, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                       

In [17]:
pred = random_search.predict(X_test)

accuracy_score(y_test, pred)

0.7127139364303179

In [18]:
random_search.best_score_

0.7182636765205572

In [19]:
random_search.best_params_

{'vect__min_df': 0.05,
 'vect__max_features': 1000,
 'vect__max_df': 0.75,
 'clf__n_estimators': 10,
 'clf__max_depth': 20}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [12]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [13]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [14]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [15]:
subNumber = 1

In [16]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

# **SUBMISSION 2**

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

### **sklearn.linear_model.SGDClassifier**
This estimator implements regularized linear models with stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate). SGD allows minibatch (online/out-of-core) learning via the partial_fit method. For best results using the default learning rate schedule, the data should have zero mean and unit variance.

This implementation works with data represented as dense or sparse arrays of floating point values for the features. The model it fits can be controlled with the loss parameter; by default, it fits a linear support vector machine (SVM).

In [17]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

sgdc = SGDClassifier()

lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', sgdc)])
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [18]:
parameters = {
    'lsi__svd__n_components': [10, 100, 150, 200, 250, 300, 400],
}

grid_search = GridSearchCV(pipe, parameters, cv=10, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed: 17.1min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
            

In [19]:
from sklearn.metrics import accuracy_score
pred = grid_search.predict(X_test)

accuracy_score(y_test, pred)

0.7444987775061125

In [20]:
grid_search.best_score_

0.7561931295848108

In [21]:
grid_search.best_params_

{'lsi__svd__n_components': 400}

### Make a Submission File

In [22]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [23]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [24]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [25]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# **SUBMISSION 3**

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here
import spacy

nlp = spacy.load("en_core_web_lg")

In [ ]:
def get_words_vectors(docs):
    return[nlp(doc).vector for doc in docs]

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?